## let's try a logistic regression model

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re
import sqlite3 as sl
np.random.seed(42)
%matplotlib inline

In [5]:
con = sl.connect('tweets')
corpus2 = pd.read_sql("""
                    SELECT tweet_text, is_there_an_emotion_directed_at_a_brand_or_product
                    FROM tweets
                    WHERE tweet_text != 'None'
                    AND is_there_an_emotion_directed_at_a_brand_or_product IN ('Positive emotion','Negative emotion')
                    """,con)
corpus2.head()  

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [9]:
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

sw = stopwords.words('english')

In [14]:
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
    
sw = stopwords.words('english')
sw.extend(["sxsw","mention","link","ipad","rt","apple","google","iphone",
                  "quot","store","app","new","get","austin","pop","amp","android","launch","go","open"] )
    
def doc_preparer(doc, stop_words=sw):
    '''  
    :param doc: a document from the corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in stop_words]
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [15]:
X = corpus2.tweet_text.astype('str')
y = corpus2.is_there_an_emotion_directed_at_a_brand_or_product

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.25)

token_docs = [doc_preparer(doc, sw) for doc in X_train]
token_docs[0]

'give away x box book techenvy'

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(X_train)
X = cv.transform(X_train)
X_test = cv.transform(X_train)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(accuracy_score(y_val,lr.predict(X_val)))

ValueError: could not convert string to float: "They're giving away iPad 2's, x boxes and books at @mention #sxsw #techenvy"